<a href="https://colab.research.google.com/github/vashchirag02/nlp_projects/blob/main/langgraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# STEP 0: Install dependencies

In [59]:
!pip install -qU google_generativeai==0.8.5 google_ai_generativelanguage==0.6.15\
langgraph langchain langchain-google-genai openai
!pip install -q google-generativeai


# STEP 1: Imports and secure API key input

In [60]:
import os
import getpass
from langgraph.graph import StateGraph,END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage


# Secure Gemini API Key input

In [61]:
os.environ["OPEN_API_KEY"] = getpass.getpass("Enter your api key")

Enter your api key··········


# STEP 2: Initialize Gemini 1.5 Flash

In [70]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.4,
    google_api_key=os.environ["OPEN_API_KEY"]
)

# STEP 3: Node to ask user for symptom

In [71]:
def get_symptom(state : dict)-> dict:
  symptom=input('Welcome to our hospital! Please enter your symptoms')
  state["symptom"]=symptom
  return state

# STEP 4: Node to classify the symptom

In [72]:

def classify_symptom(state : dict)-> dict:
  prompt=(
      "you are a helpful medical assistant , classify the symptom below into one of the categories\n"
      "General \n""Emergency \n" "Mental Health \n"
      f"symptom: {state['symptom']} \n"
      "Respond only with one word : General ,Emergency or Mental Health"
      "Example: input : I have fever ,output :General"
  )
  response = llm.invoke([HumanMessage(content=prompt)])
  category = response.content.strip()
  print(f"llm classifies the symptom as : {category}")
  state["category"]=category
  return state

# STEP 5: Router logic to route to the correct node


In [77]:
def symptom_router(state : dict)-> dict:
  cat =state["category"].lower()
  if "general" in cat:
    return "general"
  elif "emergency" in cat:
    return "emergency"
  elif "mental_health" in cat:
    return "mental_health"
  else:
    return "general"

# STEP 6: Category-specific response nodes


In [78]:
def general_node(state :dict)-> dict:
  state["answer"]=f" '{state['symptom']}' : seems general : directing you to a generl ward to consult with the doctor"
  return state

def emergency_node(state:dict)->dict:
  state["answer"]=f" '{state['symptom']}' : it is medical emergency : seeking immediate help"
  return state

def mental_health_node(state:dict)->dict:
  state["amswer"]=f"  '{state['symptom']}' : seems like a mental health issue : speak to our counellor "
  return state

# STEP 7: Build LangGraph


In [79]:
builder = StateGraph(dict)

builder.set_entry_point("get_symptom")

builder.add_node("get_symptom",get_symptom)
builder.add_node("classify",classify_symptom)
builder.add_node("general",general_node)
builder.add_node("emergency",emergency_node)
builder.add_node("mental_health",mental_health_node)

builder.add_edge("get_symptom","classify")
builder.add_conditional_edges("classify",symptom_router,{
    "general": "general",
    "emergency": "emergency",
    "mental_health": "mental_health"
})
builder.add_edge("general",END)
builder.add_edge("emergency",END)
builder.add_edge("mental_health",END)

# STEP 8: Compile and invoke the graph


In [81]:
graph =builder.compile()
final_state= graph.invoke({})
print("final output \n")
print(final_state["answer"])

Welcome to our hospital! Please enter your symptomsmy nose is bleeding
llm classifies the symptom as : Emergency
final output 

 'my nose is bleeding' : it is medical emergency : seeking immediate help
